In [1]:
import keras
from keras import layers
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, AveragePooling2D
from keras import regularizers
from keras.optimizers import Adam
import pandas as pd

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
kernel = 1.0 * RBF(1.0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
from sklearn import preprocessing


In [3]:
# load the dataset
import pandas as pd
from scipy.io import arff
data = arff.loadarff('/content/drive/MyDrive/final_paper/MLL.arff') # selcet labeled data according to application
df = pd.DataFrame(data[0])
df.shape

(72, 12583)

In [4]:
from sklearn import preprocessing
print( df.iloc[:,-1].unique()) # to check number of classes in the dataset
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
# Encode labels in column 'species'.
df.iloc[:,-1]= label_encoder.fit_transform(df.iloc[:,-1])
df.iloc[:,-1].unique()

[b'ALL' b'MLL' b'AML']


array([0, 2, 1], dtype=object)

In [5]:
# separate dataset according to classes
x1 = df[df.iloc[:,-1] == 0]
x2 = df[df.iloc[:,-1] == 1]
x3 = df[df.iloc[:,-1] == 2]
print(x1.shape,x2.shape,x3.shape) # to check number of samples in each class

(24, 12583) (28, 12583) (20, 12583)


In [6]:
#separate the target classes
Y1 = x1.iloc[:,-1]
Y2 = x2.iloc[:,-1]
Y3 = x3.iloc[:,-1]
# remove the class label
del x1[x1.columns[-1]]
del x2[x2.columns[-1]]
del x3[x3.columns[-1]]
print(x1.shape,x2.shape,x3.shape) # to check number of samples in each class without class labels

(24, 12582) (28, 12582) (20, 12582)


In [7]:
#Select samples for augmentation
n=10
df1=x1.sample(n =10)
df2=x2.sample(n = 10)
df3=x3.sample(n = 10)


In [8]:
import numpy as np
mu, sigma = 0, 0.1
# creating a noise with the same dimension as the dataset
noise1 = np.random.normal(mu, sigma, [10,x1.shape[1]])
noise2 = np.random.normal(mu, sigma, [10,x1.shape[1]])
noise3 = np.random.normal(mu, sigma, [10,x1.shape[1]])
print(noise1.shape,noise2.shape,noise3.shape) # check the shape of the noise

(10, 12582) (10, 12582) (10, 12582)


In [9]:
# add noise to selected samples
noise_factor = 0.1
DF1 =  df1+ (noise_factor*noise1)
DF2=   df2+ (noise_factor*noise2)
DF3=   df3+ (noise_factor*noise3)


In [10]:
# add noisy sample to original data
verti_df1= pd.concat([x1, DF1], axis=0)
verti_df2= pd.concat([x2, DF2], axis=0)
verti_df3= pd.concat([x3, DF3], axis=0)
print(verti_df1.shape,verti_df2.shape,verti_df3.shape)# check the shape of the augmented data

(34, 12582) (38, 12582) (30, 12582)


In [11]:
#merge all samples to new augmented data
new_data=pd.concat([verti_df1,verti_df2,verti_df3],axis=0)
new_data.shape

(102, 12582)

In [12]:
#create equal class labels
y1= Y1.sample(n = 10)
y2= Y2.sample(n = 10)
y3= Y3.sample(n = 10)

new_y1= pd.concat([Y1,y1],axis=0)
new_y2=pd.concat([Y2,y2],axis=0)
new_y3=pd.concat([Y3,y3],axis=0)
new_Class= pd.concat([new_y1,new_y2,new_y3],axis=0)
new_Class.shape

(102,)

In [13]:
#merge class label to dataset to create augmented dataset(data with label)
from sklearn.utils import shuffle
NEW_dataset=new_data.reset_index(drop=True).merge(new_Class.reset_index(drop=True), left_index=True, right_index=True)
NEW_dataset= (shuffle(NEW_dataset)).reset_index(drop=True)

In [14]:
NEW_dataset.shape

(102, 12583)

In [15]:
print(NEW_dataset.shape)
dataset= NEW_dataset.values
X = dataset[:,:-1]
Y = dataset[:,-1]
print(X.shape,Y.shape)
input_dim=X.shape[1]
print(input_dim)

(102, 12583)
(102, 12582) (102,)
12582


In [16]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
scaler = MinMaxScaler(feature_range=(0, 1))
X_scl_train = scaler.fit_transform(X_train)
X_scl_test = scaler.fit_transform(X_test)
X_scl = scaler.fit_transform(X)

In [17]:
input_data = keras.Input(shape=(input_dim,))
encoded = layers.Dense(1024, activation='relu')(input_data)
output = layers.Dense(input_dim, activation='sigmoid')(encoded)
en_m=keras.Model(input_data,encoded)
#encoder_m.summary()
auto_m = keras.Model(input_data, output)
auto_m.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense (Dense)               (None, 1024)              12884992  
                                                                 
 dense_1 (Dense)             (None, 12582)             12896550  
                                                                 
Total params: 25781542 (98.35 MB)
Trainable params: 25781542 (98.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
auto_m.compile(optimizer='adam', loss='binary_crossentropy')
auto_m.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 5s 1s/step - loss: 0.6826 - val_loss: 0.6989
Epoch 2/100
3/3 [==============================] - 1s 468ms/step - loss: 0.6378 - val_loss: 0.6693
Epoch 3/100
3/3 [==============================] - 1s 471ms/step - loss: 0.6267 - val_loss: 0.6650
Epoch 4/100
3/3 [==============================] - 1s 470ms/step - loss: 0.6202 - val_loss: 0.6553
Epoch 5/100
3/3 [==============================] - 2s 747ms/step - loss: 0.6127 - val_loss: 0.6523
Epoch 6/100
3/3 [==============================] - 2s 729ms/step - loss: 0.6069 - val_loss: 0.6498
Epoch 7/100
3/3 [==============================] - 2s 663ms/step - loss: 0.6034 - val_loss: 0.6448
Epoch 8/100
3/3 [==============================] - 1s 475ms/step - loss: 0.5999 - val_loss: 0.6409
Epoch 9/100
3/3 [==============================] - 1s 477ms/step - loss: 0.5961 - val_loss: 0.6385
Epoch 10/100
3/3 [==============================] - 1s 471ms/step - loss: 0.5928 - val_loss: 0.6367
Epoch 11/100

In [19]:
#adding two new layers autoencoder 2
en2=(Dense(512,activation='relu'))(auto_m.layers[-2].output)
de2=(Dense(1024,activation='relu'))(en2)
ou=(Dense(input_dim,activation='sigmoid'))(de2)
#creating new model
ae_model=keras.Model(inputs=auto_m.input, outputs=[ou])
encoder_model=keras.Model(inputs=auto_m.input, outputs=[en2])
encoder_model.summary()
ae_model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense (Dense)               (None, 1024)              12884992  
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
Total params: 13409792 (51.15 MB)
Trainable params: 13409792 (51.15 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12582)]           0         
                                                                 


In [20]:
#train new model
ae_model.compile(optimizer='adam', loss='binary_crossentropy')
ae_model.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 4s 845ms/step - loss: 0.6832 - val_loss: 0.6745
Epoch 2/100
3/3 [==============================] - 2s 654ms/step - loss: 0.6414 - val_loss: 0.6590
Epoch 3/100
3/3 [==============================] - 1s 484ms/step - loss: 0.6208 - val_loss: 0.6548
Epoch 4/100
3/3 [==============================] - 1s 470ms/step - loss: 0.6097 - val_loss: 0.6507
Epoch 5/100
3/3 [==============================] - 1s 476ms/step - loss: 0.5973 - val_loss: 0.6391
Epoch 6/100
3/3 [==============================] - 1s 477ms/step - loss: 0.5897 - val_loss: 0.6374
Epoch 7/100
3/3 [==============================] - 2s 892ms/step - loss: 0.5834 - val_loss: 0.6327
Epoch 8/100
3/3 [==============================] - 1s 476ms/step - loss: 0.5770 - val_loss: 0.6289
Epoch 9/100
3/3 [==============================] - 2s 753ms/step - loss: 0.5723 - val_loss: 0.6253
Epoch 10/100
3/3 [==============================] - 2s 734ms/step - loss: 0.5676 - val_loss: 0.6218
Epoch 11/

In [21]:
#adding two new layers
en3=(Dense(100,activation='relu'))(ae_model.layers[-3].output)
de3=(Dense(512,activation='relu'))(en3)
de3=(Dense(1024,activation='relu'))(de3)
ou3=(Dense(input_dim,activation='sigmoid'))(de3)

In [22]:
#creating final stacked model
ae_model_new=keras.Model(inputs=auto_m.input, outputs=[ou3])
SAE=keras.Model(inputs=auto_m.input, outputs=[ae_model_new.layers[-4].output])
ae_model_new.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 12582)]           0         
                                                                 
 dense (Dense)               (None, 1024)              12884992  
                                                                 
 dense_2 (Dense)             (None, 512)               524800    
                                                                 
 dense_5 (Dense)             (None, 100)               51300     
                                                                 
 dense_6 (Dense)             (None, 512)               51712     
                                                                 
 dense_7 (Dense)             (None, 1024)              525312    
                                                                 
 dense_8 (Dense)             (None, 12582)             1289

In [23]:
#train final stacked model
ae_model_new.compile(optimizer='adam', loss='binary_crossentropy')
ae_model_new.fit(X_scl_train, X_scl_train,
                epochs=100,
                batch_size=30,
                shuffle=True,
                validation_data=(X_scl_test, X_scl_test))

Epoch 1/100
3/3 [==============================] - 3s 558ms/step - loss: 0.6863 - val_loss: 0.6706
Epoch 2/100
3/3 [==============================] - 1s 482ms/step - loss: 0.6434 - val_loss: 0.6644
Epoch 3/100
3/3 [==============================] - 1s 467ms/step - loss: 0.6221 - val_loss: 0.6718
Epoch 4/100
3/3 [==============================] - 2s 602ms/step - loss: 0.6164 - val_loss: 0.6517
Epoch 5/100
3/3 [==============================] - 2s 802ms/step - loss: 0.6055 - val_loss: 0.6520
Epoch 6/100
3/3 [==============================] - 2s 849ms/step - loss: 0.5985 - val_loss: 0.6371
Epoch 7/100
3/3 [==============================] - 1s 475ms/step - loss: 0.5916 - val_loss: 0.6412
Epoch 8/100
3/3 [==============================] - 1s 473ms/step - loss: 0.5855 - val_loss: 0.6309
Epoch 9/100
3/3 [==============================] - 1s 470ms/step - loss: 0.5805 - val_loss: 0.6312
Epoch 10/100
3/3 [==============================] - 1s 480ms/step - loss: 0.5770 - val_loss: 0.6286
Epoch 11/

In [24]:
X_new = SAE.predict(X_scl)
print(X_new.shape)

4/4 [==============================] - 0s 37ms/step
(102, 100)


In [33]:
cv = KFold(n_splits=2, random_state=1, shuffle=True)
rf_model = RandomForestClassifier(n_estimators=500, random_state=0)
LR = LogisticRegression(random_state=0)
lda_clf = LinearDiscriminantAnalysis()
qda_clf = QuadraticDiscriminantAnalysis()
gpc = GaussianProcessClassifier(kernel=kernel,random_state=0)
svm_model = svm.SVC(random_state=1,kernel='rbf', probability=True)
gnb = GaussianNB()
dt_model = DecisionTreeClassifier(random_state=0)
adb = AdaBoostClassifier(n_estimators=100, random_state=0)
modls=[rf_model,LR,lda_clf,qda_clf,gpc,svm_model,gnb,dt_model,adb]
names=["RF", "LR", "LDA", "QDA", "GPC", "SVM", "NB", "DT", "AB"]

In [30]:
X.shape

(102, 100)

In [29]:
Y.shape

(102,)

In [36]:
#X,Y=load_dataset(df)
X=X_new
size=X.shape[1]
print(X.shape)
#ws=create_ws(WB,'sae')
#ws.write(0,0,  (size) )
print('    Accuracy: recall: precision : F1 score: kappa: AUC  ' )
for mo in range(9):
    print(names[mo])
    row=mo
    col=1
    scores = cross_val_score(modls[mo], X, Y, scoring='accuracy', cv=cv, n_jobs=-1)
    p_scores = cross_val_score(modls[mo], X, Y, scoring='precision_macro', cv=cv, n_jobs=-1)
    r_scores = cross_val_score(modls[mo], X, Y, scoring='recall_macro', cv=cv, n_jobs=-1)
    f_scores = cross_val_score(modls[mo], X, Y, scoring='f1_macro', cv=cv, n_jobs=-1)
    acc=(mean(scores)*100)
    re= (mean(r_scores)*100)
    pre=(mean(p_scores)*100)
    f1=(mean(f_scores)*100)
    print('    %.2f   %.2f   %.2f   %.2f' %(acc, re, pre, f1))


(102, 100)
    Accuracy: recall: precision : F1 score: kappa: AUC  
RF


ValueError: 
All the 2 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
2 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 385, in fit
    y, expanded_class_weight = self._validate_y_class_weight(y)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 746, in _validate_y_class_weight
    check_classification_targets(y)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/multiclass.py", line 218, in check_classification_targets
    raise ValueError("Unknown label type: %r" % y_t

In [39]:
#find the hightest weighted genes
weight = SAE.get_weights()
we_ts1=np.asarray(weight[0])
we_ts2=np.asarray(weight[2])
we_ts3=np.asarray(weight[4])
aa=np.dot(we_ts1,we_ts2)
result=np.dot(aa,we_ts3)
result.shape

(12582, 100)

In [40]:
total_gene=result.shape[0]
total_node=result.shape[1]
gene_weight_list=[]
for t_g in range(total_gene):
    weight_sum=0

    for node in range(total_node):
        weight_sum=weight_sum+result[t_g][node]

    #print(t_g)
    #print(weight_sum)
    gene_weight_list.append((t_g,weight_sum))
print(t_g)

12581


In [43]:
gene_list=sorted(gene_weight_list,key=lambda gene_weight_list: gene_weight_list[1] ,reverse=True)
top_gene=gene_list[:50]
gen_data = [lis[0] for lis in top_gene]
gene_data=np.asarray(gen_data)
col_name=[]
for col in NEW_dataset.columns:
    col_name.append(col)
gene_name=[]
for ge_name in gene_data:
    gene_name.append(col_name[ge_name])

In [44]:
#hightest weighted genes
gene_name

['1214_s_at',
 '32793_at',
 '37780_at',
 '40729_s_at',
 '40531_at',
 '1852_at',
 '34144_at',
 '31865_at',
 '33678_i_at',
 '40155_at',
 '40367_at',
 '39812_at',
 '33977_at',
 '38995_at',
 '35258_f_at',
 '33528_at',
 '455_at',
 '587_at',
 '34882_at',
 '40922_at',
 '34482_at',
 '35449_at',
 '35555_r_at',
 '37728_r_at',
 '1226_at',
 '32844_at',
 '37804_at',
 '1275_at',
 '32164_at',
 '41350_at',
 '2057_g_at',
 '40677_at',
 '31859_at',
 '1365_at',
 '32921_at',
 '36156_at',
 '38193_at',
 '39822_s_at',
 '37197_s_at',
 '39161_at',
 '41215_s_at',
 '39829_at',
 '33958_at',
 '31970_r_at',
 '33535_at',
 '39904_at',
 '38836_at',
 '1727_at',
 '36299_at',
 '33452_at']